In [6]:
from elasticsearch import Elasticsearch
import xmltodict
import pandas as pd

In [8]:
df = pd.read_csv("./data/sephora_website_dataset.csv")

In [9]:
df.head()

,id,brand,category,name,size,rating,number_of_reviews,love,price,value_price,...,MarketingFlags,MarketingFlags_content,options,details,how_to_use,ingredients,online_only,exclusive,limited_edition,limited_time_offer
0,2218774,Acqua Di Parma,Fragrance,Blu Mediterraneo MINIATURE Set,5 x 0.16oz/5mL,4.0,4,3002,66.0,75.0,...,True,online only,no options,This enchanting set comes in a specially handc...,Suggested Usage:-Fragrance is intensified by t...,Arancia di Capri Eau de Toilette: Alcohol Dena...,1,0,0,0
1,2044816,Acqua Di Parma,Cologne,Colonia,0.7 oz/ 20 mL,4.5,76,2700,66.0,66.0,...,True,online only,- 0.7 oz/ 20 mL Spray - 1.7 oz/ 50 mL Eau d...,An elegant timeless scent filled with a fresh-...,no instructions,unknown,1,0,0,0
2,1417567,Acqua Di Parma,Perfume,Arancia di Capri,5 oz/ 148 mL,4.5,26,2600,180.0,180.0,...,True,online only,- 1oz/30mL Eau de Toilette - 2.5 oz/ 74 mL E...,Fragrance Family: Fresh Scent Type: Fresh Citr...,no instructions,Alcohol Denat.- Water- Fragrance- Limonene- Li...,1,0,0,0
3,1417617,Acqua Di Parma,Perfume,Mirto di Panarea,2.5 oz/ 74 mL,4.5,23,2900,120.0,120.0,...,True,online only,- 1 oz/ 30 mL Eau de Toilette Spray - 2.5 oz/...,Panarea near Sicily is an an island suspended ...,no instructions,unknown,1,0,0,0
4,2218766,Acqua Di Parma,Fragrance,Colonia Miniature Set,5 x 0.16oz/5mL,3.5,2,943,72.0,80.0,...,True,online only,no options,The Colonia Miniature Set comes in an iconic A...,Suggested Usage:-Fragrance is intensified by t...,Colonia: Alcohol Denat.- Water- Fragrance- Lim...,1,0,0,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9168 entries, 0 to 9167
Data columns (total 21 columns):
id                        9168 non-null int64
brand                     9168 non-null object
category                  9168 non-null object
name                      9168 non-null object
size                      9168 non-null object
rating                    9168 non-null float64
number_of_reviews         9168 non-null int64
love                      9168 non-null int64
price                     9168 non-null float64
value_price               9168 non-null float64
URL                       9168 non-null object
MarketingFlags            9168 non-null bool
MarketingFlags_content    9168 non-null object
options                   9168 non-null object
details                   9168 non-null object
how_to_use                9168 non-null object
ingredients               9168 non-null object
online_only               9168 non-null int64
exclusive                 9168 non-null int64
limi

In [11]:
df.columns

Index(['id', 'brand', 'category', 'name', 'size', 'rating',
       'number_of_reviews', 'love', 'price', 'value_price', 'URL',
       'MarketingFlags', 'MarketingFlags_content', 'options', 'details',
       'how_to_use', 'ingredients', 'online_only', 'exclusive',
       'limited_edition', 'limited_time_offer'],
      dtype='object')

In [12]:
select = ['id', 'brand', 'category', 'name', 'size', 'rating',
       'number_of_reviews', 'love', 'price', 'URL', 'details',
       'how_to_use', 'ingredients']
df = df[select]

In [20]:
df.iloc[0].how_to_use

'Suggested Usage:-Fragrance is intensified by the warmth of your own body. Apply in the creases of your knees and elbows for a longer-lasting- stronger scent.\xa0-After applying- avoid rubbing or dabbing skin. This breaks down the fragrance- causing it to wear off more quickly.\xa0-If you prefer placing fragrance on your wrists- be sure to reapply after frequent hand-washing- as this tends to rinse off the scent.\xa0-Replace fragrance after 12 months. Expired perfumes more than a year old lose the integrity of the original scent. Eau de Toilette or Eau de Parfum?: The difference lies in the volume of perfume oil. While EDT contains five to nine percent- EDP contains more- usually eight to 14 percent. EDPs- therefore- last longer and smell more intense.'

In [30]:
df["body"] = (df.brand + " " + df.name + " "+ df.details +" "+ df.how_to_use)

In [31]:
df.head()

,id,brand,category,name,size,rating,number_of_reviews,love,price,URL,details,how_to_use,ingredients,body
0,2218774,Acqua Di Parma,Fragrance,Blu Mediterraneo MINIATURE Set,5 x 0.16oz/5mL,4.0,4,3002,66.0,https://www.sephora.com/product/blu-mediterran...,This enchanting set comes in a specially handc...,Suggested Usage:-Fragrance is intensified by t...,Arancia di Capri Eau de Toilette: Alcohol Dena...,Acqua Di Parma Blu Mediterraneo MINIATURE Set ...
1,2044816,Acqua Di Parma,Cologne,Colonia,0.7 oz/ 20 mL,4.5,76,2700,66.0,https://www.sephora.com/product/colonia-P16360...,An elegant timeless scent filled with a fresh-...,no instructions,unknown,Acqua Di Parma Colonia An elegant timeless sce...
2,1417567,Acqua Di Parma,Perfume,Arancia di Capri,5 oz/ 148 mL,4.5,26,2600,180.0,https://www.sephora.com/product/blu-mediterran...,Fragrance Family: Fresh Scent Type: Fresh Citr...,no instructions,Alcohol Denat.- Water- Fragrance- Limonene- Li...,Acqua Di Parma Arancia di Capri Fragrance Fami...
3,1417617,Acqua Di Parma,Perfume,Mirto di Panarea,2.5 oz/ 74 mL,4.5,23,2900,120.0,https://www.sephora.com/product/blu-mediterran...,Panarea near Sicily is an an island suspended ...,no instructions,unknown,Acqua Di Parma Mirto di Panarea Panarea near S...
4,2218766,Acqua Di Parma,Fragrance,Colonia Miniature Set,5 x 0.16oz/5mL,3.5,2,943,72.0,https://www.sephora.com/product/colonia-miniat...,The Colonia Miniature Set comes in an iconic A...,Suggested Usage:-Fragrance is intensified by t...,Colonia: Alcohol Denat.- Water- Fragrance- Lim...,Acqua Di Parma Colonia Miniature Set The Colon...


In [32]:
es = Elasticsearch()

(2218774,
 'Acqua Di Parma Blu Mediterraneo MINIATURE Set This enchanting set comes in a specially handcrafted blue box- and includes a selection of fragrances from the Blu Mediterraneo collection. A symbol of the Italian Mediterranean and the island of Capri- Arancia di Capri is sunny- relaxing- and carefree. In the air- hints of Italian citrus and the warm aroma of caramel blend together to create a pure moment of bliss- just like being on vacation.Soarkling- authentic bergamot shines at the onset of Bergamotto di Calabria. It is enhanced by the freshness of citron- red ginger- and cedarwood. At the base- an unprecedented combination of vetiver- benzoin- and musk emerges. The Amalfi Coast: it’s one of the most breathtaking places on Earth. Fico di Amalfi is a floral- woody- and citrusy fragrance that calls to mind this breathtaking stretch of Mediterranean coastline with a strong- energizing effect. Mirto di Panarea is characterized by the aromatic notes of myrtle and basil- it opens

In [45]:
def index_sephora(df):
    if es.indices.exists(index='sephora'):
        print("Index 'sephora' already exists, skip indexing.")
        return
    for i in range(df.shape[0]):
        product_id = df.iloc[i].id
        body = {"details": df.iloc[i].body,
                "ingredients": df.iloc[i].ingredients}
        res = es.index(index='sephora', id=product_id, body=body)
        

In [46]:
index_sephora(df)

Index 'sephora' already exists, skip indexing.


In [97]:
def search_query(query, size, param):
    body = {
        "from":0,
        "size": size,
        "query": {
            "match": {
                param:query
            }
        }
    }

    res = es.search(index="sephora", body=body)
    print("Top", size, "results for query:", query )
    ids = []
    scores = []
    for hit in res['hits']['hits']:
        i, score = hit['_id'], hit['_score']
        product = " | ".join(df[df.id==int(i)][["brand","name"]].values[0])
        # name = type(i)
        ids.append(i)
        scores.append(score)
        print(i, score, product, sep="\t")
        
    return ids, scores

In [98]:
query = "skincare for dry skin"
ids, scores = search_query(query, 10, "details")

Top 10 results for query: skincare for dry skin
1988716	4.930148	Kiehl's Since 1851 | Cucumber Herbal Alcohol-Free Toner
1988682	4.6511035	Kiehl's Since 1851 | Calendula Herbal Extract Alcohol Free Toner
1898535	4.602146	philosophy | Renewed Hope in a Jar Skin Tint
1989896	4.573933	Kiehl's Since 1851 | Creme de Corps Mini
1780915	4.566028	Bobbi Brown | Instant Detox Hawaiian Sea Water & White Clay Face Mask
1559061	4.5315766	Dr. Dennis Gross Skincare | All-In-One Cleanser With Toner
2024198	4.505607	GLAMGLOW | #GLITTERMASK GRAVITYMUD™ Firming Treatment Mask
2131803	4.495612	Glow Recipe | Blueberry Bounce Gentle Cleanser
2188209	4.495612	Glow Recipe | Blueberry Bounce Gentle Cleanser Mini
2222974	4.4942727	tarte | Sea the Hydration Skincare Set


In [96]:
df[df.id==1988716][["brand","name"]].values[0]

array(["Kiehl's Since 1851", 'Cucumber Herbal Alcohol-Free Toner'],
      dtype=object)

In [88]:
df.loc[df.id==1988716,'name']

4392    Cucumber Herbal Alcohol-Free Toner
Name: name, dtype: object